In [ ]:
import sys
sys.path.append("../.")

In [3]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import torch
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from gnn_powergrid.dataset.utils.graph_utils import prepare_dataset

## Prepare the dataset

In [6]:
env_name = "l2rpn_case14_sandbox"

path = pathlib.Path().resolve()
BENCH_CONFIG_PATH = path / "configs" / (env_name + ".ini")
DATA_PATH = path / "Datasets" / env_name / "DC"
LOG_PATH = path / "lips_logs.log"

benchmark = PowerGridBenchmark(benchmark_path=DATA_PATH,
                               benchmark_name="Benchmark4",#"DoNothing",
                               load_data_set=True,
                               config_path=BENCH_CONFIG_PATH,
                               log_path=LOG_PATH)

In [7]:
print(benchmark.train_dataset.size)
print(benchmark._test_dataset.size)

100000
10000


In [ ]:
device = torch.device("cpu") # or "cuda:0" if you have any GPU
train_loader, val_loader, test_loader, test_ood_loader = prepare_dataset(benchmark=benchmark, 
                                                                         batch_size=128, 
                                                                         device=device)

In [9]:
batch = next(iter(train_loader))
print(batch)

DataBatch(x=[3584, 2], edge_index=[2, 6742], edge_attr=[6742], y=[3584, 1], edge_index_no_diag=[2, 4950], edge_attr_no_diag=[4950], ybus=[3584, 28], batch=[3584], ptr=[129])


In [20]:
128*28

3584

#### Implement `np.divide` in pytorch

In [11]:
from torch_geometric.nn import MessagePassing

class GPGinput_without_NN(MessagePassing):
    """Graph Power Grid Input layer

    This is the input layer of GNN initialize the theta (voltage angles) with zeros and
    updates them through power flow equation

    """
    def __init__(self,
                 ref_node,
                 device="cpu",
                 ):
        super().__init__(aggr="add")
        self.theta = None
        self.device = device
        self.ref_node=ref_node

    def forward(self, batch):
        
        # Initialize the voltage angles (theta) with zeros
        self.theta = torch.zeros_like(batch.y, dtype=batch.y.dtype)

        # Compute a message and propagate it to each node, it does 3 steps
        # 1) It computes a message (Look at the message function below)
        # 2) It propagates the message using an aggregation (sum here)
        # 3) It calls the update function which could be Neural Network
        aggr_msg = self.propagate(batch.edge_index_no_diag,
                                  y=self.theta,
                                  edge_weights=batch.edge_attr_no_diag * 100.0
                                 )
        n_bus = batch.ybus.size()[1]
        n_sub = n_bus / 2
        # keep only the diagonal elements of the ybus 3D tensors
        ybus = batch.ybus.view(-1, n_bus, n_bus) * 100.0
        denominator = torch.hstack([ybus[i].diag() for i in range(len(ybus))]).reshape(-1,1)
        # ybus = ybus * torch.eye(*ybus.shape[-2:], device=self.device).repeat(ybus.shape[0], 1, 1)
        # denominator = ybus[ybus.nonzero(as_tuple=True)].view(-1,1)
        
        input_node_power = (batch.x[:,0] - batch.x[:,1]).view(-1,1)
        numerator = input_node_power - aggr_msg
        # out = (input_node_power - aggr_msg) / denominator
        out = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)
        out = torch.Tensor(out)

        #we impose that reference node has theta=0
        out = out.view(-1, n_bus, 1) - out.view(-1,n_bus,1)[:,self.ref_node].repeat_interleave(n_bus, 1).view(-1, n_bus, 1)
        out = out.flatten().view(-1,1)
        #we impose the not used buses to have theta=0
        out[denominator==0] = 0
        
        # impose also the unused buses to have zero thetas
        
        
        return numerator, denominator
    
    def message(self, y_j, edge_weights):
        """Compute the message that should be propagated
        
        This function compute the message (which is the multiplication of theta and 
        admittance matrix elements connecting node i to j)

        Args:
            y_j (_type_): the theta (voltage angle) value at a neighboring node j
            edge_weights (_type_): corresponding edge_weight (admittance matrix element)

        Returns:
            _type_: active powers for each neighboring node
        """
        tmp = y_j * edge_weights.view(-1,1)
        return tmp
    
    def update(self, aggr_out):
        """update function of message passing layers

        We output directly the aggreated message (sum)

        Args:
            aggr_out (_type_): the aggregated message

        Returns:
            _type_: the aggregated message
        """
        return aggr_out

In [12]:
gnn_input = GPGinput_without_NN(ref_node=0)

In [70]:
num, denom = gnn_input(batch)
print(num.shape, num.dtype)
print(denom.shape, denom.dtype)

torch.Size([3584, 1]) torch.float32
torch.Size([3584, 1]) torch.float64


In [ ]:
# numpy divide version
out_numpy = np.divide(num, denom, out=np.zeros_like(num), where=denom!=0)
print("The shape is : ", out_numpy.shape)
print("The array : \n", out_numpy)

The shape is :  (3584, 1)
The array : 
 [[ 0.03797239]
 [ 0.01980564]
 [-0.01046045]
 ...
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


In [ ]:
# one way to do with torch
out_torch = torch.divide(num, denom)
out_torch[torch.isnan(out_torch)] = 0.
print("The shape is : ", out_torch.shape)
print("The array : \n", out_torch)

The shape is :  torch.Size([3584, 1])
The array : 
 tensor([[ 0.0380],
        [ 0.0198],
        [-0.0105],
        ...,
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], dtype=torch.float64)


In [65]:
# second way to do with torch
indices = torch.where(denom.flatten()!=0.)[0]
out_torch_2 = torch.zeros_like(denom)
out_torch_2[indices] = torch.divide(num[indices], denom[indices])
print("The shape is : ", out_torch_2.shape)
print("The array : \n", out_torch_2)

The shape is :  torch.Size([3584, 1])
The array : 
 tensor([[ 0.0380],
        [ 0.0198],
        [-0.0105],
        ...,
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], dtype=torch.float64)


In [68]:
print(np.sum(out_torch.numpy() - out_numpy) < 1e-7)
print(np.sum(out_torch_2.numpy() - out_numpy) < 1e-7)

True
True
